## Notebook For Eclipse Megamovie Software Deployment ##
<i>Written by David Story </i>

___
### Description
This notebook was written to be an easy-to-use plateform for learning how to interface to AWS with Python to deploy and run software on AWS cloud computing services, particularly using AWS EC2 instances. This Notebook is primarily aimed at teaching students how to distribute software for the Eclipse Megamovie to these EC2 instances in order to create datasets for the Eclipse Megamovie project, as well as related computationally heavy research projects.

This notebook will automate a number of tasks for preparing your AWS EC2 instances and deploying software to those instances. 

This notebook will:

- Create a specified amount of instances
- Send datasets to those instances
- Send processing jobs and parameters to instances
- Run jobs
- Retrieve and store results
___
### Information on Provided Files ###
This repository contains the following folders:

   - support
   - software
   - shell
   - output
   - log 
   - keys
   - instances

___
#### support: ####
This folder provides supporting Python files that handle alot of more tedious data parsing, string concatenating, and ssh commands for accessing the AWS servers. There are scripts in this folder that generate shell commands, logs, and instance info in .csv files. 

#### software: ####
The software folder is where you put any software and supporting files that you would like to send to the AWS servers. In this note book you will define a main script that will be run on the servers, and then any files you include in the software folder will also be copied over to the servers along with your main program.

#### shell: ####
This folder holds any shell scripts that are generated by this notebook. Each shell script will be copied to its own unique EC2 instance.

#### output: ####
This folder can store outputs from the servers; however it is currently not in use. Currently this notebook saves all outputs to an AWS S3 bucket.

#### log: ####
This folder holds logs that are generated by this notebook setting the parameters for the AWS instance creations.

#### keys: ####
This folder contains copies of any SSH keys created by this notebook.

#### instances: ####
This folder contains .csv files that contain information for instances that were generated by this notebook.

___
### Dependencies
This notebook uses Python 3.7.0 and requires the following libraries:
   
   - awscli
   - boto3
   - paramiko
   
# Important Note! #

There is currently no monitoring in this notebook, you should be able to monitor the EC2 instances from the AWS EC2 console. Most importantly, <b><span style="color:red">when you are done running the servers, you need to manually stop or terminate those servers or they will stay up and it will keep charging your AWS account.</span></b> I will soon include a small script you can run that automatically closes all your servers. 

___
### Setting up AWS Config

We will be using the Boto3 API to interface Python with AWS, before we can do that we need to install the AWS CLI (Command Line Interface) from here:

https://aws.amazon.com/cli/

After you have installed the CLI, you must configure your AWS credentials on the machine that you will run this code on, the configuration process can be found here:

https://boto3.amazonaws.com/v1/documentation/api/latest/guide/quickstart.html#configuration

Essentially what you will have to do is in a terminal type:
    
    aws config

After which you will be asked for security credentials, these credentials will be avaliable on your AWS account under 
"My Security Credentials"

After this is done, you will be asked an avaliability zone, for now use: 

    us-west-2a

Finally, for your output use: 
    
    json

<b>Ensure that in the configure step you set your region to the region that your EC2 and S3 instances and buckets are at, else you will not be able to access them with the API</b> 

___

### Viewing your current EC2 Instances###

Using the Boto3 library, we are able to acquire information about our instances that are in AWS using this Python API, below we are importing the libraries we will need:

In [1]:
import sys
import boto3
import botocore
from botocore.exceptions import ClientError
import OpenSSL
from OpenSSL import crypto

import time
import os
import platform

current_region = "us-west-2a"

# Make sure to change this to support = os.getcwd() + "\\support" 
operating = platform.system()
this_cwd = str(os.getcwd())
if operating == "Windows":
    support_path = this_cwd + "\\support" 
    log_path = this_cwd + "\\log"
    instance_path = this_cwd + "\\instances"
    shell_path = this_cwd + "\\shell"
    sys.path.append(support_path)
    
elif operating == "Linux":
    support_path = this_cwd + "/support/"
    log_path = this_cwd + "/log/"
    instance_path = this_cwd + "/instances/"
    shell_path = this_cwd + "/shell/"
    sys.path.append(support_path)
    
else:
    support_path = this_cwd + "/support/"
    log_path = this_cwd + "/log/"
    instance_path = this_cwd + "/instances/"
    shell_path = this_cwd + "/shell/"
    sys.path.append(support_path)
    
# Print the following paths based on the operating system    
print(this_cwd)
print(support_path)
print(log_path)
print(shell_path)
print(instance_path)

import ecmegaresources
import ecmegacopy
import ecmegarun

C:\Users\David\Documents\GitHub\eclipse-deployment-system
C:\Users\David\Documents\GitHub\eclipse-deployment-system\support
C:\Users\David\Documents\GitHub\eclipse-deployment-system\log
C:\Users\David\Documents\GitHub\eclipse-deployment-system\shell
C:\Users\David\Documents\GitHub\eclipse-deployment-system\instances


Now that the libraries are acquired, we will use the ec2 resource to see what instances we currently have on our console:

In [2]:
# creating ec2 resource, ec2 client, and s3 connection
try:
    ec2 = boto3.resource('ec2')
    client = boto3.client('ec2')
    s3 = boto3.resource('s3')
except:
    print("Error creating clients, check AWS configuration in AWS CLI")
    sys.exit()
    
# printing instances created 
print("EC Resource:", ec2)
print("EC Client:", client)
print("S3 Connection:", s3, "\n")

# printing what running instances we have up
print("EC2 Instances Running:")

# filtering for instances by name and if running
running_instances = ec2.instances.filter(
    Filters=[{'Name': 'instance-state-name', 'Values': ['running']}])

# printing the names of running instances and their machine type
for instance in running_instances:
    print(instance.id, instance.instance_type)

# printing what stopped instances we have
print("\nEC2 Instances Stopped:")

# filtering for instances by name and if stopped
stopped_instances = ec2.instances.filter(
    Filters=[{'Name': 'instance-state-name', 'Values': ['stopped']}])

# printing the names of running instances and their machine type
for instance in stopped_instances:
    print(instance.id, instance.instance_type)

# for other cases other than running or stopped
other_instances = ec2.instances.filter(
    Filters=[{'Name': 'instance-state-name', 'Values': ['pending', 'shutting-down',
                                                    'terminated','stopping']}])

print("\nEC2 Instances Other:")
for instance in other_instances:
    print(instance.id, instance.instance_type)
    
print("\nDone Searching")

EC Resource: ec2.ServiceResource()
EC Client: <botocore.client.EC2 object at 0x0000019094180C88>
S3 Connection: s3.ServiceResource() 

EC2 Instances Running:

EC2 Instances Stopped:

EC2 Instances Other:

Done Searching


### Setting up the instances needed ###

We will now be preparing the instances that we want to process the images with.

<b> Warning: in the following process we will be stopping all of the current instances and creating a defined amount of instances that we will use for our image processing </b>

You will be asked to confirm that you want to begin the process of pausing your instances and starting the server process.

In [3]:
# Waits for user to authorize stop of their instances
go = False
while(go == False):
    user = input("Authorize to stop and store current instances ([Y]/n): ")
    if (user == "Y"):
        go = True
    elif (user == 'n'):
        print("Entered No: Exiting Program")
        sys.exit()
    else:
        print("Invalid Input:", user)

# Filters a list of all instances before we create new ones
old_instances = ec2.instances.filter(
    Filters=[{'Name': 'instance-state-name', 
              'Values': ['running', 'stopped',
                        'pending','stopping']
             }])

# Stores ID of old instances and then stops all instances
print("Stopping:")
stopIds = []
for instance in old_instances:
    instance.stop(instance.id)
    stopIds.append(instance.id)
stopIds   

Authorize to stop and store current instances ([Y]/n): Y
Stopping:


[]

### Creating new instances ###

Now that the above instances are stopped, we want to create the number of instances that we want to use. AWS by default only allows you to run 20 instances in reserve, with a more limited number on On-Demand time. This notebook is set to the 20 instance reserved limit. 

You can request more instance allocations from a link on this page: https://aws.amazon.com/ec2/faqs/#How_many_instances_can_I_run_in_Amazon_EC2

Then change the 20 on line 6 to the maximum number of instances you can use.

Input the number of instances you want to use below:

In [4]:
answer = True
while answer:
    number_of_instances = abs(int(input("Number of instances to use: ")))
    if (number_of_instances <= 0):
        print("Invalid: Enter valid number")
    elif (number_of_instances > 20):
        print("Invalid: Cannot create more than 20 Instances")
    else:
        answer = False
        
print(number_of_instances, "EC2 instances will be created.")
number_of_instances

Number of instances to use: 2
2 EC2 instances will be created.


2

We will now create a key pair that will use to SSH into our servers. We will share the same key with every server for convenience.

In [5]:
working = os.getcwd()
print(working)
try:
    os.mkdir("keys") 
except:
    print("Key path already exists")
newWorking =str(working)+"\keys"
# Creating key pair for Eclipse Megamovie usages
try:
    keyname = 'ecmega-master-key'
    newWorking = newWorking + "\\" + keyname
    key = ec2.create_key_pair(KeyName=keyname)
    newName = str(key.key_name) + ".pem"
    local_key = open(newWorking, 'w')
    local_key.write(key.key_material)
    local_key.close()
    os.chmod(newWorking, 400)
    print("Key created and saved on AWS and locally at:" ,)
except:
    print("Unable to create key, may already exist")
keyname

C:\Users\David\Documents\GitHub\eclipse-deployment-system
Key path already exists
Key created and saved on AWS and locally at:


'ecmega-master-key'

We will create a list of names for our new EC2 Instances

In [6]:
base = "ECMEGA-SERVER-"
namelist = []
for name in range(number_of_instances):
    newname = base + str(name+1)
    namelist.append(newname)
namelist

['ECMEGA-SERVER-1', 'ECMEGA-SERVER-2']

We now want to find and get an AMI (Amazon Machine Image) that is an image of a server that is already set up to run the software that we want. There will be a public image for a MegaMovie configured server on AWS. We will find it by filtering AMI images by our owner id:

In [7]:
megaImage = ec2.images.filter(Filters=[{'Name':'owner-id', 'Values':['346926079389']}])
print("Images avaliable from Eclipse MegaMovie AWS:")
count = 1
avaliableImages = []
for images in megaImage:
    print(str(count)+".",images.name, images.id, images.architecture)
    count += 1
    avaliableImages.append(images)
avaliableImages

Images avaliable from Eclipse MegaMovie AWS:
1. MegaMovie-Master-V4 ami-0bc9c19fe6120c9ea x86_64
2. MegaMovie-Master-V3 ami-0d0099f1c41c01572 x86_64


[ec2.Image(id='ami-0bc9c19fe6120c9ea'), ec2.Image(id='ami-0d0099f1c41c01572')]

Type the number printed above of the image of the server that you want to create the instances with below:

In [8]:
status = True 
length = len(avaliableImages)
while status:
    chooseImage = int(input("Enter number corresponding to image above: "))
    if chooseImage < 1: 
        print("Invalid number, enter a valid number")
    elif chooseImage > length:
        print("Invalid number, enter a valid number")
    else:
        print("Using image:", avaliableImages[chooseImage-1])
        status = False
usingImage = avaliableImages[chooseImage-1]
usingImage.name

Enter number corresponding to image above: 2
Using image: ec2.Image(id='ami-0d0099f1c41c01572')


'MegaMovie-Master-V3'

Finally, we want to choose a security group that we will assign to the servers. The security group defines who can access the server and how they can access it. There is a MegaMovie group avaliable that allows you to login and access the notebooks that are running on individual servers. We will now list the avaliable security groups below:

In [9]:
secureGroups = ec2.security_groups.all()
print("Avaliable Security Groups:")
val = 0
avaliableGroups = []
for groups in secureGroups:
    val += 1
    print(str(val) +".",groups.description, groups.id)
    avaliableGroups.append(groups)
avaliableGroups

Avaliable Security Groups:
1. launch-wizard-1 created 2019-03-11T17:14:26.303-07:00 sg-06ada7c2c749c9704
2. default VPC security group sg-47d87530


[ec2.SecurityGroup(id='sg-06ada7c2c749c9704'),
 ec2.SecurityGroup(id='sg-47d87530')]

Now enter a number to select the security group you would like to use:

In [10]:
status = True 
length = len(avaliableGroups)
while status:
    chooseGroup = int(input("Enter number corresponding to group above: "))
    if chooseGroup < 1: 
        print("Invalid number, enter a valid number")
    elif chooseGroup > length:
        print("Invalid number, enter a valid number")
    else:
        print("Using image:", avaliableGroups[chooseGroup-1])
        status = False
usingGroup = avaliableGroups[chooseGroup-1]
usingGroup.description

Enter number corresponding to group above: 2
Using image: ec2.SecurityGroup(id='sg-47d87530')


'default VPC security group'

We now have all the information we need to create the instances for our run, here are the settings that you chose to create the instances with:

In [11]:
print("- Eclipse Processing Server Information -\n")
print("Number of Instances:", number_of_instances)
print("SSH key name:", keyname)
print("Security group:", usingGroup.id)
print("AMI", usingImage.id)
print("\nServer Names:")
for name in namelist:
    print("\t"+str(name))

- Eclipse Processing Server Information -

Number of Instances: 2
SSH key name: ecmega-master-key
Security group: sg-47d87530
AMI ami-0d0099f1c41c01572

Server Names:
	ECMEGA-SERVER-1
	ECMEGA-SERVER-2


#### Confirm below that you would like to create the instances above with the given parameters ##

In [12]:
leave = True
while leave:
    confirm = input("Confirm instance creation([Y]/n): ")
    if confirm == "Y":
        leave = False
    elif confirm == "n" or confirm == "N":
        print("Exiting program")
        sys.exit()
    else:
        print("Bad input")

# Creating the instances!
server_start = time.time()
new_instances = []

for i in range(number_of_instances):
    server_instance = ec2.create_instances(ImageId=usingImage.id,
                                            KeyName=keyname,
                                            MinCount=1,
                                            MaxCount=1, 
                                            Placement={'AvailabilityZone': "us-west-2a",}, 
                                            InstanceType='t2.2xlarge', 
                                            SecurityGroupIds=[usingGroup.id], 
                                            TagSpecifications=[{'ResourceType':'instance',
                                                                'Tags': [{'Key': 'Name',
                                                                          'Value': namelist[i]}
                                                                         ,]},])
    new_instances.append(server_instance)

created_instances = []
for instances in new_instances:
    for element in instances:
        created_instances.append(element)
        
created_instances

Confirm instance creation([Y]/n): Y


[ec2.Instance(id='i-03a18e314bc7cbc80'),
 ec2.Instance(id='i-04dcaabbfab8e4f57')]

### Uploading software and datasets to the new instances

Since we now have our servers up and running, we need to transfer our scripts and data so we can begin our processing tasks.

There is a public Eclipse Megamovie S3 bucket that will contain software for different processing tasks. You can choose to use the software that is there or you can choose to use software from the local machine.

If you choose to use use S3 you will:

- Get a list of all the software in the S3 bucket
- Choose which script you want to run
- Define a timeout scenario
- Authorize that run
    
If you choose to use software on local machine:

- All software in local "Software folder" will be copied to each instance
- You will define a "main script" eg. "Servertester.py", that will be run
- Define a timeout scenario
- Authorize the run
    
More information will follow on what will happen when the scripts run and output, for now we will begin copying our information.

#### For now, choose if you will use software on S3 or software from your local machine:

Below is listed software from both S3 and local machine for choosing the main script that you want to deploy to the instances. Software on S3 is from the public bucket "software", and local machine output is from current working directory "Software" folder, if it exists.

In [13]:
directory = os.getcwd()
exists = False

print("**** LOCAL MACHINE ****\nOperating system:", operating)

for file in os.listdir(directory):
    if file == "software" and operating == "Windows":
        exists = True
        softDir = str(directory) + "\\" + "software"
    elif file == "software" and operating == "Linux":
        exists = True
        softDir = str(directory) + "/" + "software"

softList = []
if exists:
    print("Software directory at:", softDir)
    print("-------------------------------------")
    localIt = 1
    for software in os.listdir(softDir):
        softList.append(software)
        print(str(localIt)+".", software)
        localIt += 1
    
    print("-------------------------------------\n"
          + "Files in software:", len(softList))
else:
    print("Software directory does not exist")

softList

**** LOCAL MACHINE ****
Operating system: Windows
Software directory at: C:\Users\David\Documents\GitHub\eclipse-deployment-system\software
-------------------------------------
1. FastTest.py
2. ServerTesters.py
3. SingleProcessTest.py
-------------------------------------
Files in software: 3


['FastTest.py', 'ServerTesters.py', 'SingleProcessTest.py']

In [14]:
progChoice = False
while not progChoice:
        mainProg = input("Choose main program from local folder: ")
        for file in softList:
            if str(mainProg) == str(file):
                progChoice = True
        if not progChoice:
            print("Invalid file name, enter valid file name.")
print("\nUsing main program:", mainProg)

Choose main program from local folder: ServerTesters.py

Using main program: ServerTesters.py


Now that we have selected the program from our specified location, the next section will create log files to the local log folder, make .csv of the instances avaliable to the instances folder, and will create shell scripts for each server in the shell folder.

In [15]:
# creates log of server information
try:
    os.chdir(log_path)
    log_file = ecmegaresources.create_log(created_instances, "local", mainProg, softList)
    os.chdir(this_cwd)
    os.chdir(instance_path)
    instance_file = ecmegaresources.create_instance_file(created_instances)
    print(log_file)
    print(instance_file)
    os.chdir(this_cwd)
except:
    os.chdir(this_cwd)
    print("Error creating log file")
    sys.exit()

# creates shell script to run software on servers on reboot
try:
    shell_files = []
    os.chdir(shell_path)
    for i in range(len(created_instances)):
        shell = ecmegaresources.create_shell(mainProg, (i+1))
        shell_files.append(shell)
    os.chdir(this_cwd)

except:
    os.chdir(this_cwd)
    print("Error: Unable to create shell scripts.")
    
shell_files

<_io.TextIOWrapper name='ecmega-server-log-2019-03-22.txt' mode='w' encoding='cp1252'>
ecmega-instance-file-2019-03-22.csv


[<_io.TextIOWrapper name='ECMEGA-Program-Run-1.sh' mode='w' encoding='cp1252'>,
 <_io.TextIOWrapper name='ECMEGA-Program-Run-2.sh' mode='w' encoding='cp1252'>]

Now we can copy all the code to the servers and start running the programs. The servers will automatically save all the outputs the ecmega-project-bucket on S3. If you specified software from local pc, this will be transfered to the instances, else the specified software from S3 will be sent to the instance.

In [ ]:
initial_contents = ecmegacopy.local_copy_to_servers(instance_file, keyname)

### Run the programs on the Cloud

Now that we have copied all of the files to the servers we can now run our program and get back our results! The shell script on the server will automatically tar and send the files from the server to an S3 bucket.

In [ ]:
ecmegarun.run_server(instance_file, keyname)